In [1]:
import os
import pandas as pd
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
cwd = os.getcwd()
path = cwd + "\output"
if not os.path.exists(path):
    os.mkdir(path)

df = pd.read_csv('taxa4FDR.txt', delimiter = "\t")
y = list(df.iloc[:, 0])
df.drop([df.columns[0]], axis=1, inplace=True)
df = df.transpose()
df = df.loc[(df!=0).any(axis=1)]
groupsize = (len(y)/2)

# First calcuate p-value from Mann-Whitney U rank test on two independent samples
dfMWUdata = pd.DataFrame()
indexlist = []
MWUstatlist = []
MWUpvallist = []

for index, row in df.iterrows():
    row = pd.Series(row)
    controls = []
    cases = []
    for i in range(int(groupsize)):
        cases.append(row[i])
        controls.append(row[i+groupsize])
    MWUstat, MWUpval = mannwhitneyu(cases, controls, method="exact")
    indexlist.append(index)
    MWUstatlist.append(MWUstat)
    MWUpvallist.append(MWUpval)
    print(index, 'with a p-value of', MWUpval)

dfMWUdata['Index'] = indexlist
dfMWUdata['MWUstat'] = MWUstatlist
dfMWUdata['MWUpval'] = MWUpvallist

# Time to adjust p-values - for options see https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
reject, correctedpvals, _, correctedalpha = multipletests(MWUpvallist, alpha=0.05, method='bonferroni', is_sorted=False, returnsorted=False)
print()
#print(np.sum(reject))
print('The corrected alpha is ' + str(correctedalpha))
dfMWUdata['Correctedpvals'] = correctedpvals
dfMWUdata.to_csv('output\dfMWUdata.txt', sep='\t', index=False)


sns.kdeplot(MWUpvallist, color="red", shade=True, label='raw')
ax = sns.kdeplot(correctedpvals, color="green", shade=True, label='adujusted')
ax.set(xlim=(0, 1))
plt.title('Change in p-value Distribution')
plt.xlabel('p-values')
plt.ylabel('Density')
plt.legend(loc = 'best')
plt.show()

TypeError: mannwhitneyu() got an unexpected keyword argument 'method'

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
s__Candidatus_Babela_massiliensis,1.270000e-06,0.000002,0.000002,0.000010,0.000004,0.000015,0.000002,0.000003,0.000001,0.000006,...,0.000006,0.000008,4.060000e-06,0.000005,0.000002,0.000005,0.000001,2.410000e-06,0.000003,0.000002
s__Thermobaculum_terrenum,1.130000e-01,0.113000,0.113000,0.113000,0.113000,0.113000,0.113000,0.113000,0.113000,0.113000,...,0.000006,0.000008,9.970000e-07,0.000004,0.000003,0.000004,0.000003,9.650000e-07,0.000005,0.000004
s__Vampirococcus_sp._LiM,8.440000e-07,0.000002,0.000003,0.000009,0.000055,0.000013,0.000006,0.000008,0.000009,0.000014,...,0.000017,0.000027,5.630000e-06,0.000009,0.000007,0.000012,0.000005,9.450000e-06,0.000008,0.000029
s__Acidisarcina_polymorpha,3.450000e-06,0.000003,0.000007,0.000022,0.000019,0.000013,0.000014,0.000003,0.000009,0.000010,...,0.000018,0.000017,2.250000e-05,0.000011,0.000011,0.000015,0.000007,9.840000e-06,0.000010,0.000037
s__Acidobacterium_capsulatum,2.530000e-06,0.000003,0.000003,0.000025,0.000057,0.000021,0.000014,0.000013,0.000020,0.000027,...,0.000025,0.000022,5.410000e-06,0.000012,0.000011,0.000017,0.000006,1.130000e-05,0.000013,0.000030
